In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Load the dataset
file_path = '../BigBasket Products.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [2]:
# Checking for missing values
missing_values = data.isnull().sum()

# Checking for duplicate entries
duplicate_entries = data.duplicated().sum()

# Overview of the rating distribution
rating_distribution = data['rating'].describe()

missing_values, duplicate_entries, rating_distribution


(index              0
 product            1
 category           0
 sub_category       0
 brand              1
 sale_price         0
 market_price       0
 type               0
 rating          8626
 description      115
 dtype: int64,
 0,
 count    18929.000000
 mean         3.943410
 std          0.739063
 min          1.000000
 25%          3.700000
 50%          4.100000
 75%          4.300000
 max          5.000000
 Name: rating, dtype: float64)

In [3]:

# Drop rows with missing 'rating' as they are critical for popularity-based recommendations
data.dropna(subset=['rating'], inplace=True)

categorical_features = ['category','sub_category','brand']
label_encoders = {}

for feature in categorical_features:
    le = LabelEncoder()
    data[feature] = le.fit_transform(data[feature].astype(str))  # Use 'le' instead of 'encoder'
    label_encoders[feature] = le

data['description'] = data['description'].astype(str)

# Now, proceed with tokenizing the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['description'])
text_sequences = tokenizer.texts_to_sequences(data['description'])
text_padded = pad_sequences(text_sequences, maxlen=100)

# Normalize numerical features
scaler = MinMaxScaler()
data['rating'] = scaler.fit_transform(data[['rating']])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(text_padded, data['rating'], test_size=0.2, random_state=42)


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Dropout

input_dim = text_padded.shape[1]  # This is the maxlen of your text sequences

# Define the encoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)

# Define the decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)

# Compile the autoencoder
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Summary of the model
autoencoder.summary()


2024-02-09 05:38:06.147509: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-09 05:38:06.148760: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-09 05:38:06.148770: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-09 05:38:06.150262: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-09 05:38:06.150890: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 128)               12928     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 100)               12900     
                                                                 
Total params: 46564 (181.89 KB)
Trainable params: 46564 (181.

In [5]:
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)


Epoch 1/50


2024-02-09 05:38:06.842047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


48/48 [==============================] - 1s 12ms/step - loss: -184084656.0000 - val_loss: -1263619200.0000
Epoch 2/50
48/48 [==============================] - 0s 7ms/step - loss: -18208225280.0000 - val_loss: -68522754048.0000
Epoch 3/50
48/48 [==============================] - 0s 7ms/step - loss: -309768060928.0000 - val_loss: -821679226880.0000
Epoch 4/50
48/48 [==============================] - 0s 7ms/step - loss: -2280959770624.0000 - val_loss: -4797118808064.0000
Epoch 5/50
48/48 [==============================] - 0s 7ms/step - loss: -10241957691392.0000 - val_loss: -18385010163712.0000
Epoch 6/50
48/48 [==============================] - 0s 7ms/step - loss: -33159523598336.0000 - val_loss: -53824865697792.0000
Epoch 7/50
48/48 [==============================] - 0s 7ms/step - loss: -87332279549952.0000 - val_loss: -131075405774848.0000
Epoch 8/50
48/48 [==============================] - 0s 7ms/step - loss: -197824012615680.0000 - val_loss: -279861578956800.0000
Epoch 9/50
48/48 [==

In [6]:
# Extract the encoder part of the autoencoder
encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-3].output)  # Assuming -3 is the encoded layer

# Generate embeddings for the entire dataset
item_embeddings = encoder_model.predict(text_padded)


592/592 [==============================] - 1s 899us/step


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(item_embeddings)

def recommend_items(item_id, similarity_matrix, items, label_encoders, top_k=3):
    # Fetch the category of the specified item
    item_category = items.iloc[item_id]['category']

    # Get similarity scores for the item
    similarity_scores = list(enumerate(similarity_matrix[item_id]))

    # Sort the items based on the similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Initialize a list to hold indices of items that match the category constraint
    filtered_indices = []

    # Iterate over sorted similarity scores and filter by category
    for idx, score in similarity_scores:
        if items.iloc[idx]['category'] == item_category and idx != item_id:
            filtered_indices.append(idx)
        if len(filtered_indices) == top_k:
            break

    # Fetch the recommended items
    recommended_items = items.iloc[filtered_indices]

    # Decode the categorical features for readability
    for feature in ['category', 'sub_category', 'brand']:
        recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])

    # Sort the recommended items in descending order of rating
    recommended_items_sorted = recommended_items.sort_values(by='rating', ascending=False)

    return recommended_items_sorted


In [8]:
# Updated example usage
recommended_items = recommend_items(6, similarity_matrix, data, label_encoders, top_k=3)
print(recommended_items[['product', 'category', 'sub_category', 'brand', 'rating','sale_price']])


                                                product          category  \
6240                      Body Deodorant - Forest Spice  Beauty & Hygiene   
1539  Like An Ocean Breeze On A Warm Summer Day Room...  Beauty & Hygiene   
2730                      Body Deodorant - Hydra Energy  Beauty & Hygiene   

           sub_category       brand  rating  sale_price  
6240     Men's Grooming  Wild Stone   0.425       139.3  
1539  Fragrances & Deos      Nyassa   0.375       475.0  
2730  Fragrances & Deos  Wild Stone   0.000       139.3  


/var/folders/fp/tqnwc7rd219_w0rxtd8b8pz00000gn/T/ipykernel_74485/1232832943.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])
/var/folders/fp/tqnwc7rd219_w0rxtd8b8pz00000gn/T/ipykernel_74485/1232832943.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])
/var/folders/fp/tqnwc7rd219_w0rxtd8b8pz00000gn/T/ipykern

In [9]:
# Evaluate the model on the test set
test_loss = autoencoder.evaluate(X_test, X_test)
print(f'Test Loss: {test_loss}')


119/119 [==============================] - 0s 3ms/step - loss: -2324926407903281152.0000
Test Loss: -2.324926407903281e+18


In [10]:
item_0_data = data.iloc[0]
item_0_data

index                                                           1
product                    Garlic Oil - Vegetarian Capsule 500 mg
category                                                        2
sub_category                                                   42
brand                                                        1652
sale_price                                                  220.0
market_price                                                220.0
type                                             Hair Oil & Serum
rating                                                      0.775
description     This Product contains Garlic Oil that is known...
Name: 0, dtype: object

In [11]:
from tensorflow.keras.models import load_model
import joblib
import pickle


In [12]:
# Load the autoencoder model
autoencoder = load_model('../Code/Models/autoencoder_model.h5')

# Load the encoder model
encoder_model = load_model('../Code/Models/encoder_model.h5')

# Load the tokenizer
with open('../Code/Models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the label encoders
label_encoders = joblib.load('../Code/Models/label_encoders.joblib')

# Load the scaler
scaler = joblib.load('../Code/Models/scaler.joblib')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
def get_recommendations_for_product(product_name, data, similarity_matrix, label_encoders, top_k=3):
    # Find the index of the product
    product_indices = data[data['product'].str.contains(product_name, case=False, na=False)].index
    if not product_indices.empty:
        product_index = product_indices[0]
    else:
        return f"No product found matching name '{product_name}'"

    # Generate recommendations
    recommended_items = recommend_items(product_index, similarity_matrix, data, label_encoders, top_k=top_k)

    # Return recommended product details
    return recommended_items[['product', 'category', 'sub_category', 'brand', 'rating','sale_price']]

# Example usage
product_name = 'Skin Oil'
recommendations = get_recommendations_for_product(product_name, data, similarity_matrix, label_encoders, top_k=3)
print(recommendations)


                                 product          category       sub_category  \
1412  Herbal Amla & Reetha Hair Cleanser  Beauty & Hygiene          Hair Care   
4114       Shallaki - Tablets (Wellness)  Beauty & Hygiene  Health & Medicine   
2322     Fragrance Body Spray - Tranquil  Beauty & Hygiene  Fragrances & Deos   

             brand  rating  sale_price  
1412  Love In Life   0.875      160.00  
4114        Jovees   0.750      136.00  
2322  Remy Marquis   0.600      189.05  


/var/folders/fp/tqnwc7rd219_w0rxtd8b8pz00000gn/T/ipykernel_74485/1232832943.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])
/var/folders/fp/tqnwc7rd219_w0rxtd8b8pz00000gn/T/ipykernel_74485/1232832943.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_items[feature] = label_encoders[feature].inverse_transform(recommended_items[feature])
/var/folders/fp/tqnwc7rd219_w0rxtd8b8pz00000gn/T/ipykern